<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;"><h1 align="center">Сборный проект "Исследование лога событий и А/А/В-тест изменения шрифта в приложении"</h1></div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
<h3>Основная цель работы:</h3>
    <p>
Изучить поведение пользователей мобильного приложения сервиса по продаже продуктов питания.
<p>
<h3>Ключевые задачи исследования:</h3>
<ul>
    <li>Изучение событийной воронки продаж с целью определения "пробуксовывающих" этапов, формирование рекомендаций по улучшению конверсии пользователей внутри воронки;
    <li>Изучение поведения пользователя после замены шрифта в приложении. Расчет уровня значимости различий в поведении пользователя, рекомендации по введению или не введению изменений.
</ul>
<p>
<h3>Этапы работы:</h3>
<ol>
<li> Знакомство и подготовка полученных данных к работе;
<li> Формирование событийной воронки продаж, поиск этапа, на котором теряется наибольшее количество пользователей;
<li> Проведение А/А теста как подготовка к запуску А/А/В-теста по исследованию изменения шрифта. Проверка корректности работы механизма расчета теста.
<li> Анализ результатов А/А/В теста с целью проверки необходимости изменения шрифта в приложении;
<li> Формирование итоговых выводов и рекомендаций.
</ol>
</div>

## Знакомство с данными и предварительная их обработка

In [1]:
#импорт библиотек для работы
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as st
import math as mth
from plotly import graph_objects as go

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
    Имеется файл с журналом событий мобильного приложения. 
    <p>Также в файле есть указание на отношение записи к той или иной группе проведенного А/А/В теста.
    <p>Аннотация:
    <ul>
<li> EventName — название события;
<li> DeviceIDHash — уникальный идентификатор пользователя;
<li> EventTimestamp — время события;
<li> ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.
</ul>

In [7]:
#загрузка данных
try:    
    logs = pd.read_csv('/Users/macbookair/Desktop/Практикум/logs_exp.csv', sep='\t')
except FileNotFoundError:
    logs = pd.read_csv('/datasets/logs_exp.csv', sep='\t')

In [8]:
#знакомство с датасетами
def data_info(data):
    display(data.head(20)), data.info(), display(data.describe()), print(data.duplicated().sum())
data_info(logs)

,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
5,CartScreenAppear,6217807653094995999,1564055323,248
6,OffersScreenAppear,8351860793733343758,1564066242,246
7,MainScreenAppear,5682100281902512875,1564085677,246
8,MainScreenAppear,1850981295691852772,1564086702,247
9,MainScreenAppear,5407636962369102641,1564112112,246


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


,DeviceIDHash,EventTimestamp,ExpId
count,2.441260e+05,2.441260e+05,244126.000000
mean,4.627568e+18,1.564914e+09,247.022296
std,2.642425e+18,1.771343e+05,0.824434
min,6.888747e+15,1.564030e+09,246.000000
25%,2.372212e+18,1.564757e+09,246.000000
50%,4.623192e+18,1.564919e+09,247.000000
75%,6.932517e+18,1.565075e+09,248.000000
max,9.222603e+18,1.565213e+09,248.000000


413


<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Первичная проверка данных показала следующее:
   <ul>
       <li> Отсуствуют пропуски в данных, имеем чуть более 244 тысяч записей
       <li> Есть 413 строк - явных дубликатов
       <li> Столбец с датой и временем необходимо перевести в читаемый формат
       <li> Названия столбцов неудобны для работы, необходимо изменить
       <li> Наименования групп эксперимента неудобны для работы. Чтобы оценивать информацию было удобно наглядно, изменим цифровые значения на строковые
       <li> Строковые названия событий необходимо дополнительно проверить на наличие скрытых дубликатов
   </ul>
</div>

In [4]:
#в первую очередь удаляем дубликаты
logs=logs.drop_duplicates()

In [5]:
#изменение названия столбцов (сознательно без применения str.lower(), чтобы задать более понятные названия)
logs = logs.rename(
    columns={'EventName':'event_name', 'DeviceIDHash':'user_id', 'EventTimestamp':'event_timestamp', 'ExpId':'group'}
)

In [6]:
#добавляем столбец с датой и временем в читаемом формате, и для дальнейших вычислений, также в формате даты
logs['event_datetime'] = pd.to_datetime(logs['event_timestamp'], unit='s')
logs['event_date'] = logs['event_datetime'].dt.date

KeyError: 'event_timestamp'

In [ ]:
#изменим названия групп эксперимента на более понятные
logs['group'] = logs['group'].map({246:'A1', 247: 'A2', 248: 'B'})

In [ ]:
#проверяем строковый столбец на наличие неявных дубликатов
print(logs['event_name'].unique())

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Получили 5 названий событий:
    <ul>
        <li> MainScreenAppear - главный экран
        <li> PaymentScreenSuccessful - экран успешной оплаты
        <li> CartScreenAppear - экран заказа (корзины)
        <li> OffersScreenAppear - экран предложения
        <li> Tutorial - экран обучения
     </ul>
 </div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Осталось проверить даты в представленном файле, и выяснить, нет ли "пробелов" в записи данных. Если они есть, то необходимо выделить непрерывный отрезок, когда записывались абсолютно все события абсолютно всех пользователей. Если этого не сделать - может получиться неравномерная и неправдивая картина поведения пользователей.</div>

In [ ]:
#строим гистограмму по столбцу с датой и временем
logs['event_datetime'].hist(bins=14, color='CadetBlue')
plt.xticks(logs['event_date'].unique(), rotation=30)
plt.title('Гистограмма распределения количества логов по дням')
plt.ylabel('Количество записей')
plt.show()

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Наглядно видно, что до 01-08-2019 данные о событиях неполные. Поэтому в анализе необходимо оставить только диапазон дат 01-08-2019 по 07-08-2019.
<p>
После формирования отфильтрованного фрейма стоит проверить, какие потери произошли в данных других колонок, и достаточно ли их будет для анализа:</div>

In [ ]:
#оставляем в работе фрейм с отфильтрованными датами
date = dt.date(2019,8,1)
logs_filtered = logs.query('event_date >= @date')

In [ ]:
#проверяем, какие потери в данных после фильтрации, и все ли категории на месте
print(
    'Процент потерянных записей после фильтрации: {:.2%}'
    .format(1-(logs_filtered.shape[0]/logs.shape[0]))
)
print(
    'Процент потерянных пользователей после фильтрации: {:.2%}'
    .format(1-(logs_filtered['user_id'].nunique()/logs['user_id'].nunique()))
)
print(
    'Количество наименований событий до фильтрации {} и после {}'
    .format(logs['event_name'].nunique(), logs_filtered['event_name'].nunique())
)
print(
    'Количество групп до фильтрации {} и после {}'
    .format(logs['group'].nunique(), logs_filtered['group'].nunique())
)

In [ ]:
#посмотрим, какое количество данных осталось после фильтрации
print(
    'Количество событий в логе: {}'
    .format(logs_filtered['event_name'].count())
)
print(
    'Количество пользователей: {}'
    .format(logs_filtered['user_id'].nunique())
)
print(
    'Среднее количество событий на пользователя {:.2f}'
    .format(logs_filtered['event_name'].count()/logs_filtered['user_id'].nunique())
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Потери в рамках допустимых минимальных значений, и все категориальные данные сохранены. Можно переходить к детальному изучению.
</div>

## Формирование событийной воронки продаж и ее анализ

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Событийная воронка необходима для понимания действий пользователя и анализа, на каком этапе возникает отсеивание пользователей, не дошедших до этапа покупки.<p>
Стоит определить, есть ли взаимосвязанные события, которые выстраиваются в цепочку продаж, как часто встречается то или иное событие в цепочке, сколько пользователей побывало на том или ином событии и какой процент переходит на каждый новый этап цепочки.</div>

In [ ]:
#формируем таблицу с названиями событий и количеством раз, которые они произошли
events_count = logs_filtered['event_name'].value_counts()
display(events_count)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
    Заметно, что событие "Tutorial" произошло значительно меньше раз, чем остальные. Это логично, ведь обучение необходимо не всем пользователям. Также понятно, что страница Tutorial точно не входит в событийную воронку.
</div>

In [ ]:
#формируем таблицу с названиями событий и количеством пользователей, совершивших это событие
users_per_event = (
    logs_filtered.groupby('event_name').agg({'user_id': 'nunique'})
    .sort_values(by='user_id', ascending=False)
    .reset_index()
)
#переименовываем колонки
users_per_event.columns = ['event_name', 'users_count']
#считаем процент пользователей от общего числа, совершивших событие
users_per_event['percent_of_total_users'] = round(users_per_event['users_count']/logs_filtered['user_id'].nunique()*100,2)
display(users_per_event)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
    Видно, что не все пользователи открыли приложение с первого шага. На стартовом этапе у нас всего 98.47 % пользователей. И также видно, что всего 46.97% пользователей от общего числа сделали заказ (дошли до экрана успешной оплаты). Вероятно, потеря 1,5% на первом событии связана либо с неполной картиной событийного ряда (например, не записались события открытия первого экрана), либо с тем, что некоторые пользователи могут заходить в приложение через страницы, отсылающие сразу к предложению (пуш-уведомления со скидкой на товар, или баннер на стороннем сайте с определенным товаром).<p>
    Для наглядности уберем из списка событий открытие страницы с обучением, и добавим процент потери на каждом этапе:
</div>

<div class = 'alert alert-info'>
    <h4>Комментарий студента:</h4>
    Дорогой ревьюер, изначально я написала вот такой код для добавления процента потери пользователей при переходе с этапа на этап. Но потом в жарких обсуждениях с коллегами увидела великолепный метод shift, о котором не знала, и решила использовать его, ведь это значительно сокращает запись. Но дабы не попасть на обвинения в плагиате, прикрепляю то, что было изначально:
    </div>

```
users_per_event['stage_loss'] = pd.Series(0,range(len(users_per_event)))
for i in range(len(users_per_event)):
    try:
        users_per_event.loc[i,'stage_loss'] = (
            round((1-users_per_event.loc[i,'users_count']/users_per_event.loc[i-1, 'users_count'])*100,2)
        )
    except:
        users_per_event.loc[i, 'stage_loss'] = 0
````

In [ ]:
#убираем экран с обучением из воронки
users_per_event = users_per_event[users_per_event['event_name'] != "Tutorial"]
#добавляем процент потери на каждом этапе
users_per_event['stage_loss'] = round(
    (1- (users_per_event['users_count'] / users_per_event['users_count'].shift()))*100, 2
).fillna(0)
#расчитываем процент пользователей, прошедших с главного экрана до оплаты
payers_percent = (round(
    int(users_per_event[users_per_event['event_name'] == "PaymentScreenSuccessful"]['users_count']) / \
    int(users_per_event[users_per_event['event_name'] == "MainScreenAppear"]['users_count']) *100,2)
)
display(users_per_event)
print('Процент пользователей, перешедших с первого события до оплаты {}'.format(payers_percent))

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Процент пользователей, сконвертировавшихся в покупателей внутри воронки немного выше - 47.7%. Однако явно есть, что улучшать.</div>

In [ ]:
#строим воронку событий с указанием количества пользователей на каждом шаге
fig = go.Figure(go.Funnel(x=users_per_event['users_count'], y=users_per_event['event_name'], textposition = "inside",
    textinfo = "value+percent previous", marker = {"color": "Teal"}))
fig.update_layout(title={
                        'text': "Воронка событий по всем пользователям",
                        'y':0.95,
                        'x':0.59,
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                  yaxis_title="Название события",
                  margin=dict(l=0, r=0, t=50, b=20))
fig.show()

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
  На визуализации воронки, и в значениях потери этапа видно, что основная потеря пользователей происходит после главного экрана. Почти 40% пользователей не переходят дальше. Однозначно, следует оценить, что именно происходит на этом этапе подробнее.
</div>

In [ ]:
#строим таблицу с временем первого события в каждом дне для пользователей
first_events = (
    logs_filtered.sort_values(by=['user_id', 'event_datetime'])
    .pivot_table(index=['user_id','event_date'], columns='event_name', values='event_datetime', aggfunc='first')
).drop(columns='Tutorial')

first_events = first_events[['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful']]

#рассчитываем длительность пребывания на каждом этапе
first_events['mainscreen_time'] = first_events['OffersScreenAppear'] - first_events['MainScreenAppear']
first_events['offerscreen_time'] = first_events['CartScreenAppear'] - first_events['OffersScreenAppear']
first_events['cartscreen_time'] = first_events['PaymentScreenSuccessful'] - first_events['CartScreenAppear']

display(first_events)

#для пользователей, совершивших переходы, считаем среднюю длительность нахождения на каждом этапе
print(
    'Среднее время просмотра главного экрана: {}, экрана с предложением: {}, корзины: {}'
    .format(first_events['mainscreen_time'].mean(), \
            first_events['offerscreen_time'].mean(), first_events['cartscreen_time'].mean())
)


<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
    Интересный момент, что среди сконвертировавшихся на следующий этап пользователей, среднее время первого просмотра главного экрана составило 23 минуты. Следует запросить данные о длительности сессий, и посмотреть эту величину детальнее еще и у пользователей, ушедших с этого этапа.<p>
    Работа с данным пунктом невозможна в дальнейшем исслеовании в рамках работы, но была бы весьма полезна для интернет-магазина
</div>

In [ ]:
#строим сводную таблицу количества событий для каждого пользователя
events_per_user = (
    logs_filtered
    .pivot_table(index='user_id', columns='event_name', values='event_datetime', aggfunc='count')
    .fillna(0)
).drop(columns='Tutorial')

events_per_user = events_per_user[['MainScreenAppear', 'OffersScreenAppear', \
                                   'CartScreenAppear', 'PaymentScreenSuccessful']]

display(events_per_user)

#выделяем пользователей, которые не прошли дальше главного экрана
only_mainscreen_users = (
    events_per_user
    .query('MainScreenAppear != 0 and CartScreenAppear == 0 \
    and OffersScreenAppear == 0 and PaymentScreenSuccessful == 0')
)

#считаем среднее количество заходов на главный экран
print(
    'Среднее количество заходов на главный экран у пользователей, совершивших только это действие: {:.2f}'
    .format(only_mainscreen_users['MainScreenAppear'].mean())
)
#и какой процент неперешедших пользователей сделал более одной попытки
print(
    'Процент пользователей, у которых более одного захода на главный экран, совершивших только это действие: {:.2%}'
    .format(only_mainscreen_users[only_mainscreen_users['MainScreenAppear']>1].shape[0] / \
            only_mainscreen_users.shape[0])
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
    Весьма показательные цифры: неперешедшие дальше главного экрана пользователи предпринимали в среднем 15 попыток зайти в приложение, и более 95% таких пользователей сделали более одной попытки зайти в приложение. <p>
    Здесь мы обнаружили возможность дополнительного исследования, но предоставленные данные на текущий момент, не позволяют провести детальный анализ.<p>
    Можно сделать заметку, что стоит оценить интуитивную понятность главного экрана, разработать методы привлечения ушедших пользователей, протестировать некоторые изменения интерфейса.
</div>

## Проведение А/А теста как подготовка к запуску А/А/В-теста по исследованию изменения шрифта. Проверка корректности работы механизма расчета теста.

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
Для того, чтобы оценить корректность проводимого теста, стоит проверить следующие критерии в контрольных группах А:
<ol>
    <li> Все пользователи должны быть равномерно распределены по группам. Рекомендуемый порог "перевеса" - 1%
    <li> Не должно быть пользователей, попавших в две и более групп
    <li> Не должно быть статистически значимого различия основных показателей в контрольных группах
    </ol></div>

In [ ]:
#проверка на попадание одних и тех же пользователей в разные группы
logs_filtered.groupby('user_id').agg({'group':'nunique'}).query('group >1')

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Пользователей, попавших в две и более группы - нет</div>

In [ ]:
#проверка на равномерность наполнения групп
print(
    'Количество пользователей в группе А1: {}'.format(logs_filtered.query('group == "A1"')['user_id'].nunique())
)
print(
    'Количество пользователей в группе А2: {}'.format(logs_filtered.query('group == "A2"')['user_id'].nunique())
)
print(
    'Количество пользователей в группе В: {}'.format(logs_filtered.query('group == "B"')['user_id'].nunique())
)
print(
    'Разница в количестве пользователей между группами А1 и А2: {:.2%}'
    .format(1-(logs_filtered.query('group == "A1"')['user_id'].nunique()/ \
               logs_filtered.query('group == "A2"')['user_id'].nunique()))
)
print(
    'Разница в количестве пользователей между группами А1 и B: {:.2%}'
    .format(1-(logs_filtered.query('group == "A1"')['user_id'].nunique()/ \
               logs_filtered.query('group == "B"')['user_id'].nunique()))
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
Разница между группами А1 и А2 составляет 1.15%, а между группой А1 и В - 2.09%. Безусловно, это выше рекомендуемого значения. В идеальных условиях, стоило откорректировать процесс записи результатов теста, настроить показы определенным образом, чтобы распределение было в допустимых рамках. Однако превышение не слишком высокое, и нет возможности на текущем этапе сделать корректировку сбора данных, поэтому продолжим анализ.</div>

In [ ]:
#сформируем таблицы с названиями событий и количеством пользователей, совершивших это событие
#чтобы трижды не копировать код, напишем функцию
def users_per_event_frame(data):
    users_per_event_frame = (
        data.groupby('event_name').agg({'user_id': 'nunique'})
        .sort_values(by='user_id', ascending=False)
        .reset_index()
    )
    users_per_event_frame['total_users'] = data['user_id'].nunique()
    users_per_event_frame.columns = ['event_name', 'users_count', 'total_users']
    users_per_event_frame = users_per_event_frame[users_per_event_frame['event_name'] != "Tutorial"]
    display(users_per_event_frame)
    return users_per_event_frame

users_per_event_a1 = users_per_event_frame(logs_filtered.query('group == "A1"'))
users_per_event_a2 = users_per_event_frame(logs_filtered.query('group == "A2"'))
users_per_event_b = users_per_event_frame(logs_filtered.query('group == "B"'))

In [ ]:
#строим общую воронку по трем группам
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'A1',
    y = users_per_event_a1['event_name'],
    x = users_per_event_a1['users_count'],
    marker = {"color": "DarkCyan"},
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'A2',
    y = users_per_event_a2['event_name'],
    x = users_per_event_a2['users_count'],
    opacity = 0.65,
    marker = {"color": "Cyan"},
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'B',
    y = users_per_event_b['event_name'],
    x = users_per_event_b['users_count'],
    marker = {"color": "DarkTurquoise"},
    textinfo = "value+percent previous"))

fig.update_layout(title={
                        'text': "Воронка событий по группам пользователей",
                        'y':0.95,
                        'x':0.55,
                        'xanchor': 'center',
                        'yanchor': 'middle'},
                  yaxis_title="Название события",
                  margin=dict(l=0, r=0, t=50, b=20))

fig.show()

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Общая воронка по трем группам выглядит примерно равной, однако видно, что из этапа в этап пользователи группы А1 конвертируются чуть лучше остальных. Стоит проверить, есть ли статистически значимая разница доли пользоателей, перешедших на каждое из событий, между двумя контрольными группами А1 и А2. Если есть - мы не можем продолжать исследование, так как это будет означать, что поведение пользователей в одном и том же интерфейсе различно, и сравнивать поведение с группой, которая видит измененый шрифт, неразумно.</div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
Сделаем z-тест для оценки статистически значимой разницы между долями пользователей. <p>
Будем рассчитывать для каждой группы долю пользователей, перешедших на каждое событие, относительно общего количества пользователей в группе.<p>
Сформулируем нулевую гипотезу:  <p>
<blockquote>Различие между долями пользователей отсуствует</blockquote>
Сформулируем альтернативную гипотезу:  <p>
<blockquote>Различие между долями существует</blockquote>
Формулировка альтернативной гипотезы не уточняет направленность различия - отрицательное или положительное. Поэтому тест имеет двусторонний характер. Это важно учитывать при расчете p-value, значения, подсвечивающего результат теста. <p>
<b>Если p-value будет больше заданного уровня значимости, то отвергать нулевую гипотезу нельзя. Если же p-value меньше уровня значимости, то нулевая гипотеза отвергается в пользу альтернативной. Значит статистически значимая разница между группами есть.</b><p>
Уровень значимости стоит выставить на уровне 5%, помимо этого стит применить метод Холма для корректировки уровня значимости при множественном эксперименте.</div>

In [ ]:
#напишем функцию, чтобы сократить вставки с кодом:
def z_test_proportion(group1, group2, alpha):
    #делаем цикл внутри функции, чтобы сразу сделать проверку по всем событиям внутри групп
    for i in range(len(group1)):

        results = [int(group1.loc[i, 'users_count']), int(group2.loc[i, 'users_count'])]
        trials = [int(group1.loc[i, 'total_users']), int(group2.loc[i, 'total_users'])]

    #пропорция в первой группе:
        p1 = results[0]/trials[0]

    #пропорция во второй группе:
        p2 = results[1]/trials[1]

    #пропорция в комбинированном датасете:
        p_combined = (results[0] + results[1]) / (trials[0] + trials[1])

    #разница пропорций в датасетах
        difference = p1 - p2 

    #считаем z_value
        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

    #преобразовываем z_value в p-value, умножая на 2 для двустороннего теста
        p_value = st.norm.sf(abs(z_value))*2

    #задаем уровень значимости с поправкой на количество проводимых тестов (используем метод Холма)
        alpha_in_test = alpha/(len(group1) -  i)

    #выполняем проверку
        print(
            'Для события {} p-значение: {:.5f} при заданной alpha {:.5f}'
            .format(group1.loc[i, 'event_name'], p_value, alpha_in_test)
        )

        if p_value < alpha_in_test:
            print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
        else:
            print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными'
            ) 

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Проверяем статистически значимую разницу доли пользователей, перешедших на каждый из этапов, для контрольных групп А1 и А2:</div>

In [ ]:
#запускаем функцию, передавая ей датафреймы с группами а1 и а2
z_test_proportion(
    users_per_event_a1,
    users_per_event_a2,
    0.05
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Z-тест показал отсутствие статистически значимой разницы между контрольными группами. Значит можно продолжить анализ и сравнение с группой В.</div>

## Анализ результатов А/А/В теста с целью проверки необходимости изменения шрифта в приложении.

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Теперь необходимо запустить оценку статистически значимой разницы в следующих парах групп:
<ol>
    <li> Группа А1 и группа В
    <li> Группа А2 и группа В
    <li> Объединенная группа А (А1+А2) и группа В
</ol>
В случае обнаружения статистически значимой разницы в долях пользователей, перешедших на каждый этап воронки, стоит отдельно оценить направленность - уменьшилась или увеличилась эта доля в сравнении с контрольной группой.<p>
Если же при проверке окажется, что статистически значимая разница отсусттвует, то A/A/B тест можно завершать с указанием, что изменение шрифта никак не отразилось на поведении пользователей внутри приложения.</div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Проверяем группы А1 и В:</div>

In [ ]:
#запускаем функцию, передавая ей датафреймы с группами A1 и B
z_test_proportion(
    users_per_event_a1,
    users_per_event_b,
    0.05
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Статистически значимое различие отсуствует. Пользователи групп А1 и В конвертируются на каждый этап одинаково.</div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Проверяем группы А2 и В:</div>

In [ ]:
#запускаем функцию, передавая ей датафреймы с группами A2 и B
z_test_proportion(
    users_per_event_a2,
    users_per_event_b,
    0.05
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Статистически значимое различие отсуствует. Пользователи групп А2 и В конвертируются на каждый этап одинаково.</div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Проверяем группу А(А1+А2) и В:</div>

In [ ]:
#запускаем функцию, передавая ей датафреймы с группами A1+A2 и B
z_test_proportion(
    users_per_event_a1+users_per_event_a2,
    users_per_event_b,
    0.05
)

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">Статистически значимое различие отсуствует. Пользователи групп А и В конвертируются на каждый этап одинаково.</div>

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">По итогу проведения всех z-тестов можно однозначно сказать, что разница в конвертации пользователей в каждый из этапов воронки между группами отсуствует.<p>
    Новый шрифт в приложении не улучшил, и не ухудшил ситуацию.
    </div>

## Итоговые выводы и рекомендации.

<div style="border: solid LightSeaGreen 2px; padding: 10px; font-size: 16px;">
    Были рассмотрены данные лога с событиями пользователей внутри мобильного приложения.<p>
    Выявлена воронка событий с перемещением пользователей внутри приложения по 4 этапам:
    <ul>
        <li> MainScreenAppear
        <li> OffersScreenAppear
        <li> CartScreenAppear
        <li> PaymentScreenSuccessful
    </ul>
    При оценке событийной воронки по общим данным было выявлено:
    <ul>
        <li> Не все пользователи переходят сразу на первый этап. Около 1,5% попадают на другие экраны. Этот пункт требует дополнительного изучения - расширения лога, вероятно не все сессии записались.
        <li> 38% пользователей теряется после события MainScreenAppear, и не переходят на следующий этап, при этом такие пользователи в среднем предпринимают 15 попыток захода в приложение, и 95,5% из них - более одного раза. Здесь необходимо провести отдельный анализ, оценить причину, по которой такое большое число пользователей не переходят на этап просмотра экрана OffersScreenAppear.
        <li> Еще 18% пользователей отсеивается до перехода на этап CartScreenAppear, этот показатель также стоит протестировать и проанализировать отдельно.
        <li> Общий процент пользователей, сконвертировавшихся внутри воронки с первого до последнего этапа (от главного экрана до успешной оплаты) - 47.7%. Этот показатель однозначно можно улучшить, проведя исследование потерь на этапах MainScreenAppear и OffersScreenAppear.
    </ul>
<p>
    <h4>Далее были рассмотрены результаты проведенного А/А/В тестирования на изменение шрифта в приложении. Имеются следующие выводы:</h4>
    <ul>
        <li> При записи результатов, различие в количестве пользователей между двумя контрольными группами составило 1,15%, а между контрольными и экспериментальной - чуть более 2%. Этот показатель говорит о небольшом искажении результатов, однако не противоречит последующему продолжению анализа.
        <li> Сравнение двух контрольных групп пользователей показало <b>отсутствие разницы в поведении пользователей и прохождении от этапа к этапу внутри событийной воронки</b>.
        <li> Сравнение контрольных групп пользователей и экспериментальной также показало <b>отсутствие разницы в поведении пользователей и прохождении от этапа к этапу внутри событийной воронки</b>.
    </ul>
    <b>По итогу рассмотрения всех данных, можно признать тест оконченным и различие между группами невыявленным. Изменение или сохранение действующего шрифта остается на усмотрение менеджеров.</b>
    </div>